In [1]:
%load_ext autoreload
%autoreload 2

# Importar entorno y familiarizarse

In [2]:
from boardgame2 import ReversiEnv
import numpy as np

# Crear 3 tipos de jugador
- Random: Selecciona uniformemente una de las acciones válidas
- Greedy: Selecciona la acción que le da más ganancia inmediata (cantidad de piezas que come). Si hay más de una acción que da máxima ganancia samplear uniformemente entre ellas
- Optimum (solo para 4x4): Usando resultados de la PI optima obtenida por policy iteration

Tener en cuenta que:
- ReversiEnv tiene los métodos get_valid y next_step y no es necesario mantener el estado del entorno
- env.PASS ([-1,  0]) es una acción valida posible y debería hacerse cuando no get_valid devuelve una matriz de ceros

Para el optimo en 4x4 bajar usar la PI obtenida en la notebook anterior guardado en /mdp

In [9]:
class GreedyPlayer():
    def __init__(self, player=1, board_shape=None, env=None, flatten_action=False):
        if (env is None) and (board_shape is None):
            print("board_shape and env can't be both None")
        if env is None:
            env = ReversiEnv(board_shape=board_shape)
        self.env = env
        self.player = player # player number. 1 o -1
        self.flatten_action = flatten_action
        self.board_shape = self.env.board.shape[0]
    
    def predict(self, board):
        # defino las acciones validas
        acciones_validas = np.argwhere(self.env.get_valid((board, self.player)) == 1)
        # Si no hay opciones validas paso
        if len(acciones_validas) == 0:
            action = self.env.PASS
            print("pass")
        else:
            puntajes_jugadas = []
            for a in acciones_validas:
                proximo_estado, _, _, _ = self.env.next_step((board, self.player), a)
                puntajes_jugadas.append(proximo_estado[0].sum() * self.player)
            mejor_puntaje = max(puntajes_jugadas)
            mejores_jugadas = acciones_validas[np.array(puntajes_jugadas)==mejor_puntaje]
            #Samplea entre las mejores jugadas
            action = mejores_jugadas[np.random.randint(len(mejores_jugadas))]
        
        if self.flatten_action:
            return action[0] * self.board_shape + action[1]
        else:
            return action
        
class RandomPlayer():
    def __init__(self, player=1, board_shape=None, env=None, flatten_action=False):
        if (env is None) and (board_shape is None):
            print("board_shape and env can't be both None")
        if env is None:
            env = ReversiEnv(board_shape=board_shape)
        self.env = env
        self.player = player
        self.flatten_action = flatten_action
        self.board_shape = self.env.board.shape[0]
    
    def predict(self, board):
        # defino las acciones validas
        acciones_validas = np.argwhere(self.env.get_valid((board, self.player)) == 1)
        # Si no hay opciones validas paso
        if len(acciones_validas) == 0:
            action = self.env.PASS
            print("pass")
        else:
            # Muestrea aleatoriamente las acciones válidas
            action = acciones_validas[np.random.randint(len(acciones_validas))]
        if self.flatten_action:
            return action[0] * self.board_shape + action[1]
        else:
            return action
        

class DictPolicyPlayer():
    def __init__(self, player=1, board_shape=4, env=None, flatten_action=False, dict_folder='mdp/pi_mdp.npy'):
        self.pi_dict = np.load(dict_folder, allow_pickle=True).item()
        if env is None:
            env = ReversiEnv(board_shape=board_shape)
        self.player = player
        self.flatten_action = flatten_action
        self.board_shape = board_shape
    
    def predict(self, board):
        # Elegir la acción optima y devolverla
        board_tuple = tuple((board * self.player).reshape(-1))
        action = self.pi_dict[board_tuple]
        return action
        if self.flatten_action:
            return action
        else:
            #Formatea la salida a fila col
            return [action // self.board_shape, action % self.board_shape]

In [10]:
gp = GreedyPlayer(player=1, board_shape=4)
rp = RandomPlayer(player=1, board_shape=4)

# Verificar que el pass funciona OK

In [11]:
board = np.array([
    [-1, 0, 0, 0],
    [0, 1, 1, 0],
    [0, 1, 1, 0],
    [0, 0, 0, 0]]
)
rp.predict(board)
gp.predict(board)

pass
pass


array([-1,  0])

# Completar la función que dado dos jugadores imprima estadísticas de las partidas

In [12]:
def arena_stats(Player_1, Player_2, board_shape, N=500):
    
    env = ReversiEnv(board_shape=board_shape)
    wins_as_first = 0
    wins_as_second = 0
    plays_as_first = 0
    plays_as_second = 0
    total_steps = 0
    player_1 = Player_1(player=1, board_shape=board_shape, flatten_action=False)
    player_2 = Player_2(player=1, board_shape=board_shape, flatten_action=False)
    for i in range(N):
        # Aveces empieza un jugador, a veces el otro
        first_player = np.random.choice([-1, 1])
        player_1.player = first_player
        player_2.player = -first_player
        
        plays_as_first = plays_as_first + (first_player == 1)
        plays_as_second = plays_as_second + (first_player == -1)
        
        done = False
        n_steps = 0
        (board, player) = env.reset()
        
        while not done:
            if first_player == player:
                action = player_1.predict(board)
            else:
                action = player_2.predict(board)
            (board, player), reward, done, info = env.step(action)
            n_steps = n_steps + 1
        total_steps = total_steps + n_steps
        wins_as_first = wins_as_first + (reward == first_player) * (first_player == 1)
        wins_as_second = wins_as_second + (reward == first_player) * (first_player == -1)
    print(f'Wins as first: {wins_as_first/plays_as_first}')
    print(f'Wins as second: {wins_as_second/plays_as_second}')
    print(f'Plays as first: {plays_as_first}')
    print(f'Plays as second: {plays_as_second}')
    print(f'Avg game duration: {total_steps/N}')
        
    

In [13]:
arena_stats(DictPolicyPlayer, GreedyPlayer, 4, N=2000)

Wins as first: 0.8274428274428275
Wins as second: 1.0
Plays as first: 962
Plays as second: 1038
Avg game duration: 11.731


In [14]:
arena_stats(DictPolicyPlayer, RandomPlayer, 4, N=1000)

Wins as first: 0.8187250996015937
Wins as second: 1.0
Plays as first: 502
Plays as second: 498
Avg game duration: 11.668


In [15]:
arena_stats(RandomPlayer, DictPolicyPlayer, 4, N=1000)

Wins as first: 0.0
Wins as second: 0.1590909090909091
Plays as first: 472
Plays as second: 528
Avg game duration: 11.663


In [16]:
arena_stats(RandomPlayer, GreedyPlayer, 4, N=1000)

Wins as first: 0.4356060606060606
Wins as second: 0.559322033898305
Plays as first: 528
Plays as second: 472
Avg game duration: 11.668


In [17]:
arena_stats(RandomPlayer, RandomPlayer, 4)

Wins as first: 0.3384615384615385
Wins as second: 0.5333333333333333
Plays as first: 260
Plays as second: 240
Avg game duration: 11.824


In [18]:
arena_stats(GreedyPlayer, GreedyPlayer, 4)

Wins as first: 0.4659090909090909
Wins as second: 0.461864406779661
Plays as first: 264
Plays as second: 236
Avg game duration: 11.478


In [19]:
arena_stats(RandomPlayer, GreedyPlayer, 8, N=1000)

Wins as first: 0.35655737704918034
Wins as second: 0.3125
Plays as first: 488
Plays as second: 512
Avg game duration: 58.178


# Guardar todas las clases de jugadores en un player.py para que luego se puedan importar de la siguiente forma:

from players import RandomPlayer

from players import GreedyPlayer

# Programación Dinámica Cuestionario

## Respuesta a los ejercicios de laboratorio (continuacion)

### 6)

arena_stats(DictPolicyPlayer, RandomPlayer, 4, N=1000)

Wins as second: 1.0

#### gana 100% veces

arena_stats(DictPolicyPlayer, RandomPlayer, 4, N=1000)

Wins as first: 0.8187250996015937

#### gana cerca del 80% de las veces

arena_stats(DictPolicyPlayer, GreedyPlayer, 4, N=2000)

Wins as first: 0.8274428274428275

#### gana cerca del 80% de las veces

arena_stats(RandomPlayer, GreedyPlayer, 8, N=1000)

Wins as first: 0.35655737704918034

Wins as second: 0.3125


#### porcentaje de empates:

In [24]:
(1-(0.35655737704918034 + 0.3125))

0.33094262295081966